In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)
#train_op = tf.train.AdamOptimizer(learning_rate=0.0.1).minimize(cost, global_step=global_step) 한 줄로 되는 지 확인

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [4]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables()) #tf.global_variables 앞서 정의한 변수를 가져오는 함수

In [5]:
# 기존에 학습한 모델이 있는 지 확인
# saver.restore: 학습된 값을 불러옴
ckpt = tf.train.get_checkpoint_state('./model')

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-2


In [6]:
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('step: %d, ' % sess.run(global_step), 'cost: %.3f ' % sess.run(cost, feed_dict={X: x_data,Y: y_data}))

step: 3,  cost: 0.797 
step: 4,  cost: 0.771 


In [7]:
saver.save(sess, './model/dnn.ckpt', global_step = global_step)

'./model/dnn.ckpt-4'

불러오기 저장은 끝났고 앞장처럼 예측결과와 정확도 확인하는 코드로 마무리

In [8]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('prediction: ', sess.run(prediction, feed_dict={X: x_data, Y: y_data}))
print('real value(target): ', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('accuracy: %.2f' % sess.run(accuracy*100, feed_dict={X: x_data, Y: y_data}))

prediction:  [0 1 1 0 0 2]
real value(target):  [0 1 2 0 0 2]
accuracy: 83.33
